In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

In [ ]:
df1=pd.read_pickle("results_percolation_ALL.pickle")
df2 =pd.read_pickle("results_percolation_002.pickle")

In [ ]:
df = pd.concat([df1,df2], ignore_index = True )
df.head()

In [ ]:
df.phi.unique()
df.sort_values(['ptype','temperature','phi','delta','current_time'])

In [ ]:

grouped1 = df1.sort_values(['temperature','phi','delta']).groupby(['ptype','delta','phi'])
grouped2 = df2.sort_values(['temperature','phi','delta']).groupby(['ptype','delta','phi'])

phi=0.01
fig,(ax1,ax2) = plt.subplots(1,2)
ptype = 'double_manta_asymm_1'
delta = '0.2'
phis=[0.1,0.2,0.3]
for (g1,g2) in zip(grouped1,grouped2):
    
    if g1[0][0] != ptype or g1[0][1] != delta:
        fig,(ax1,ax2) = plt.subplots(1,2)
        ptype = g1[0][0]
        delta = g1[0][1]
   
    plt.title('{},{}'.format(g1[0][0],g1[0][1]))

    if g1[0][2] in phis:
        ax1.plot(g1[1].temperature, g1[1].frac_largest_virtual, label='$\phi =$ {}'.format(g1[0][2]))
        ax1.set_ylim([0,1.1])
        
        ax2.plot(g2[1].temperature, g2[1].frac_largest_virtual, label='$\phi =$ {}'.format(g2[0][2]))
        ax2.set_ylim([0,1.1])
    
    
    plt.legend(loc='best', bbox_to_anchor=(0.8, 0.5, 0.5, 0.5))

plt.show()
    

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import matplotlib as mpl
plt.style.use("seaborn")

df2 = df2.sort_values(['ptype', 'delta','temperature','phi'])

ptypes = df2.ptype.unique()
deltas = df2.delta.unique()

phis = df2.phi.unique()
temps = df2.temperature.unique()



for pi in ptypes:
    for di in deltas:
        fig,ax = plt.subplots(figsize=(10,10))
        arr = df2[(df2['ptype'] == pi) & (df2['delta']== di)]['frac_largest_virtual'].values
        arr = np.reshape(arr, (len(temps),len(phis)))
        
        arr = np.round(arr, decimals=2)
        mpl.rc("legend", fontsize=50)
        heat_map = sns.heatmap(arr,linewidth=1, annot=True, cmap='rainbow')
        plt.title("{},$\delta$ = {}".format(pi,di))
        plt.xlabel("$\phi$")
        plt.ylabel("Temperature")
        
        new_xticks = phis
        plt.xticks(np.arange(0,len(phis)),new_xticks, rotation=45, horizontalalignment='right')
        
        new_yticks = temps
        plt.yticks(np.arange(0,len(temps)),new_yticks)
        heat_map.invert_yaxis()
             
            
                           
                   
        
plt.show()

In [ ]:
np.arange(10)

### Representative networks 
double_manta_asymm_1 
double_mouse_asymm_1
double_mouse_symm_1
double_mouse_symm_2

phi=0.125, delta=0.2, T=0.01

data files:
psi_op.dat
patch_network.dat

Calculate:
average degree
degree distribution 
comunities, 
loops 
draw network with p,np bonds 

In [ ]:
import networkx as nx 
import numpy as np 

In [ ]:
def read_bonds(filen):
	first_line_pair = [0,0,0,0]
	cut=False
	with open(filen, 'r') as f:
		network_list = []
		for line in f:
			if "#" in line:
				network_list.append([])
				first_line_pair = [0,0,0,0]
				cut=False

			else:
				line_counter=len(network_list[-1])
				pairs = list(map(int, line.split(" ")))
				if pairs == first_line_pair or cut==True:
					cut=True
				else:
					network_list[-1].append(np.array(pairs))

				if line_counter == 0:
					first_line_pair = pairs
	network_list = [ np.array(item) for item in network_list]

	return network_list


In [ ]:
def read_network(filedir, ptype, phi,delta,T):
    
    dirn="{}/{}_phi_{}_delta_{}_temp_{}".format(filedir,ptype,phi,delta,T)
    filen="{}/patch_network.dat".format(dirn)
    
    connections = read_bonds(filen)[-1]
    print(connections)
    
    G=nx.Graph()
    G.add_edges_from(connections[:,:2])
    
    return G

    
    

In [ ]:
filedir="/Users/ada/Documents/Code_Development_2020/rhombi/percolations_study/vsc3/percolation_runs/copy_dir"
ptype='double_mouse_asymm_1'
phi=0.125
delta=0.2
T=0.01 
G=read_network(filedir,ptype,phi,delta,T)

degree_sequence=[ d for n,d in G.degree()]
average_degree = np.mean(degree_sequence)

largest_cc = max(nx.connected_components(G), key=len)
S = G.subgraph(largest_cc).copy()

